In [1]:
import os
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# import time for timekeeping
import time
import random
random.seed(time.time())

from rembg import remove

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:

def convertImages(number_of_output,input_apple,input_greenapple,input_orange,input_mango,input_capsicum,input_background ,output_image,output_label):
    start = time.time()
    
    # create output folder if it doesn't exist already
    if not os.path.exists(output_image):
        os.makedirs(output_image)
    if not os.path.exists(output_label):
        os.makedirs(output_label)
        
    ########################################################################################   
    
    cwd = os.getcwd()
    bg_list = os.listdir(input_background)
    
    apple_list = os.listdir(input_apple)
    greenapple_list = os.listdir(input_greenapple)
    orange_list = os.listdir(input_orange)
    mango_list = os.listdir(input_mango)
    capsicum_list = os.listdir(input_capsicum)
    
    # fruit_file = [apple_list,greenapple_list,orange_list,mango_list,capsicum_list]
    fruit_list = [['apple',len(apple_list)], ['greenapple',len(greenapple_list)],['orange',len(orange_list)],['mango',len(mango_list)],['capsicum',len(capsicum_list)] ]
    
    num_of_fruit = random.randint(1, 3)
    
    ## for loop for number of pictures to generate   
    for pic_num in range(number_of_output):
        # obtain pictures
        index_apple = random.randrange(1,len(apple_list)-1)        
        index_greenapple = random.randrange(1,len(greenapple_list)-1)        
        index_orange = random.randrange(1,len(orange_list)-1)        
        index_mango = random.randrange(1,len(mango_list)-1)        
        index_capsicum = random.randrange(1,len(capsicum_list)-1)
        
        index_bg = random.randrange(1,len(bg_list)-1)

        filename_bg = bg_list[index_bg]
        
        placed_fruits = set()
        f = open(os.path.join(output_label,f"image_{pic_num}.txt"), "w+") 
        if filename_bg.endswith(".png") or filename_bg.endswith('jpg'):
            img_path_bg = os.path.join(input_background, filename_bg)
            bg = Image.open(img_path_bg)
            
            for m in range(random.randint(1, 3)):
                while True:
                    # print('loop')
                    fruit_type = random.randint(0, 4)
                    if fruit_type not in placed_fruits:
                        placed_fruits.add(fruit_type)
                        break
                        
                i = random.randint(0, fruit_list[fruit_type][1]-1) # Choose a random picture of the selected fruit
                # print(str(fruit_type) + "  "+str(i))
                
                if(fruit_type ==0):
                    fruit_path = os.path.join(input_apple,  apple_list[i])
                elif(fruit_type ==1):
                    fruit_path = os.path.join(input_greenapple,  greenapple_list[i])
                elif(fruit_type ==2):
                    fruit_path = os.path.join(input_orange,  orange_list[i])
                elif(fruit_type ==3):
                    fruit_path = os.path.join(input_mango,  mango_list[i])
                elif(fruit_type ==4):
                    fruit_path = os.path.join(input_capsicum,  capsicum_list[i])
                
                fruit = Image.open(fruit_path)
                # imgplot = plt.imshow(fruit)
                # plt.show()
                
                alpha = fruit.getchannel('A')

                # Get the non-zero (non-transparent) pixels
                non_transparent_pixels = alpha.point(lambda p: p > 0, '1')

                # Get the bounding box of the non-transparent pixels
                bbox = non_transparent_pixels.getbbox()
                
                # Crop the image to the bounding box
                cropped_image = fruit.crop(bbox)
                fruit = cropped_image
                # imgplot = plt.imshow(fruit)
                # plt.show()
                
                
                imgx = bg 
                
                
                bg_width, bg_height = imgx.size
                width, height = fruit.size
                aspect = width / height
                x = np.ceil(bg_width / num_of_fruit)
                
                upper_thres = np.floor(aspect * bg_height)
                if upper_thres > x:
                    upper_thres = x
                random_width = random.randint(50, upper_thres)
                random_height = np.floor(random_width / aspect)
                new_size = (int(random_width), int(random_height))
                img4 = fruit.resize(new_size)

                random_location = (random.randint(x * m, x * m + 1), random.randint(0, bg_height - random_height))
                c1 = (random_location[0] + random_width / 2) / bg_width
                c2 = (random_location[1] + random_height / 2) / bg_height
                s1 = random_width / bg_width
                s2 = random_height / bg_height

                imgx.paste(img4, random_location, mask=img4)
                f.write(f"{str(fruit_type+1)} {c1} {c2} {s1} {s2}\n")
                # imgx.save(f"C:\\Users\\sshya\\Desktop\\Uni_2023\\ECE4078\\output\\img_{n}.jpg") 
                # imgx.save(f"D:\Monash\Y3S1\ECE4078\ECE4078-G3\milestone4\images_filter\5_NN_Ready\images\img{n}.png") 
                
                # f.write(f"{str(fruit_type)} {c1} {c2} {s1} {s2}\n")
                
                extension = ".png"
                offset = 0
                save_filename_fruit = os.path.join(output_image, "image_"  + str(pic_num+offset)+ extension)
                imgx.save(save_filename_fruit, "PNG")
                
            if((pic_num%100==0) or pic_num==(number_of_output-1)):
                end = time.time()
                print("Saved", save_filename_fruit,",  Saved", save_filename_fruit,",  time taken: " , round((end - start) / 60,3), "min") # time in minutes
                
                
        f.close()
                
            
def bbox(im):
    a = np.array(im)[:,:,:3]  # keep RGB only
    m = np.any(a != [255, 255, 255], axis=2)
    coords = np.argwhere(m)
    y0, x0, y1, x1 = *np.min(coords, axis=0), *np.max(coords, axis=0)
    return (x0, y0, x1+1, y1+1)

            
            
            
            

In [3]:

start = time.time()

convertImages(number_of_output =10000, 
              input_apple      = "./2_removedbg/apple/", 
              input_greenapple      = "./2_removedbg/greenapple/", 
              input_orange      = "./2_removedbg/orange/", 
              input_mango     = "./2_removedbg/mango/", 
              input_capsicum      = "./2_removedbg/capsicum/", 
              
              input_background = "./4_arena_pics/" ,
              output_image     = "5_NN_Ready/images",
              output_label     = "5_NN_Ready/labels")

end = time.time()
print((end - start) / 60) # time in minutes

ValueError: non-integer stop for randrange()